In [1]:
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(tidyr))
suppressPackageStartupMessages(library(tibble))
suppressPackageStartupMessages(library(stringr))
library(reshape2)
library(data.table)
library(parallel)
library(purrr)
source('/home//jupyter/BRI_Figures_Final_V2/helper_function/helper_function_IHA.r')


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘data.table’


The following objects are masked from ‘package:reshape2’:

    dcast, melt


The following objects are masked from ‘package:dplyr’:

    between, first, last



Attaching package: ‘purrr’


The following object is masked from ‘package:data.table’:

    transpose




# Read Meta Data

In [2]:
meta_data_selected_RNA <- read.csv('/home/jupyter/BRI_Figures_Final_V1/Dataset/scRNA_meta_data-2024-05-09.csv')

meta_data_selected_RNA$sample.drawDate <- as.Date(meta_data_selected_RNA$sample.drawDate)

meta_data_selected_RNA <- meta_data_selected_RNA %>%
  mutate(Flu_Year = case_when(
    sample.drawDate <= as.Date('2020-07-01') ~ '2019-2020',
    sample.drawDate > as.Date('2020-07-01') & sample.drawDate <= as.Date('2021-07-01') ~ '2020-2021',
    sample.drawDate > as.Date('2021-07-01') & sample.drawDate <= as.Date('2022-07-01') ~ '2021-2022',
    sample.drawDate > as.Date('2022-07-01') & sample.drawDate <= as.Date('2023-07-01') ~ '2022-2023',
    TRUE ~ NA_character_
  ))

meta_data_selected_RNA <- meta_data_selected_RNA %>%
  mutate(Flu_Day = case_when(
    sample.visitName	%in% c("Flu Year 1 Day 0","Flu Year 2 Day 0") ~ 'Day 0',
    sample.visitName	%in% c("Flu Year 1 Day 7","Flu Year 2 Day 7") ~ 'Day 7',
    sample.visitName	%in% c("Flu Year 1 Day 90","Flu Year 2 Day 90") ~ 'Day 90',
    TRUE ~ sample.visitName
  ))

In [3]:
meta_data_selected_RNA_D0_D7<-meta_data_selected_RNA %>% filter(Flu_Year%in% c("2021-2022","2020-2021")) %>% filter(Flu_Day %in% c("Day 0",'Day 7'))

In [4]:
selected_donor<-as.data.frame(table(meta_data_selected_RNA_D0_D7$subject.subjectGuid)) %>% filter(Freq==4)

In [5]:
meta_data_selected<-meta_data_selected_RNA_D0_D7 %>% filter(subject.subjectGuid %in% selected_donor$Var1)
write.csv(meta_data_selected,"selected_samples_with_acutal_flu_year.csv")

In [6]:
meta_data_selected_RNA_D0_D7<-meta_data_selected_RNA_D0_D7 %>% filter(subject.subjectGuid %in% selected_donor$Var1,Flu_Year%in% c("2020-2021"))

# Read Freq Files and do subsetting

In [7]:
df_freq<-read.csv("/home/jupyter/BRI_Figures_Final_V2/Dataset/diha_AIFI_L3_frequencies_2024-05-05.csv")

In [8]:
df_freq_Y1<-df_freq %>% filter(specimen.specimenGuid %in% meta_data_selected_RNA_D0_D7$pbmc_sample_id)

In [9]:
df_freq_Y1<-df_freq_Y1 %>% filter(AIFI_L3 %in% c("CD27+ effector B cell", 
                    "CD27- effector B cell", 
                    "Core memory B cell", 
                    "Plasma cell"))

In [10]:
df_freq_Y1<-left_join(df_freq_Y1,meta_data_selected_RNA_D0_D7[c('pbmc_sample_id','Flu_Year','Flu_Day')],by=c("specimen.specimenGuid"="pbmc_sample_id"))

In [13]:
write.csv(df_freq_Y1,"freq_actual_flu_year_1_D0_D7.csv")